# Calculate Table Size

```sql
CREATE TABLE #tablespaceinfo
(
  nameinfo VARCHAR(50),
  rowsinfo BIGINT,
  reserved VARCHAR(20),
  datainfo VARCHAR(20),
  index_size VARCHAR(20),
  unused VARCHAR(20)
)

DECLARE @tablename VARCHAR(255);

DECLARE Info_cursor CURSOR
FOR
    SELECT  '[' + [name] + ']'
    FROM    sys.tables
    WHERE   type = 'U';

OPEN Info_cursor FETCH NEXT FROM Info_cursor INTO @tablename
WHILE @@FETCH_STATUS = 0
BEGIN
    INSERT INTO #tablespaceinfo
        EXEC sp_spaceused @tablename
FETCH NEXT FROM Info_cursor
INTO @tablename
END

CLOSE Info_cursor
DEALLOCATE Info_cursor

CREATE TABLE [#tmptb]
(
    TableName VARCHAR(50) ,
    DataInfo BIGINT ,
    RowsInfo BIGINT ,
    Spaceperrow AS ( CASE RowsInfo WHEN 0 THEN 0 ELSE DataInfo / RowsInfo END ) PERSISTED
)

INSERT  INTO [#tmptb] ([TableName],[DataInfo], [RowsInfo])
    SELECT  [nameinfo], CAST(REPLACE([datainfo], 'KB', '') AS BIGINT) AS 'datainfo',
            [rowsinfo]
    FROM    #tablespaceinfo
    ORDER BY CAST(REPLACE(reserved, 'KB', '') AS INT) DESC

SELECT  [tbspinfo].* , [tmptb].[Spaceperrow] AS 'Row Size（KB）'
FROM    [#tablespaceinfo] AS tbspinfo, [#tmptb] AS tmptb
WHERE   [tbspinfo].[nameinfo] = [tmptb].[TableName]
ORDER BY CAST(REPLACE([tbspinfo].[reserved], 'KB', '') AS INT) DESC

DROP TABLE [#tablespaceinfo]
DROP TABLE [#tmptb]
```

# sp_who

```sql
CREATE TABLE #sp_who2 (SPID INT,Status VARCHAR(255),
      Login  VARCHAR(255),HostName  VARCHAR(255),
      BlkBy  VARCHAR(255),DBName  VARCHAR(255),
      Command VARCHAR(255),CPUTime INT,
      DiskIO INT,LastBatch VARCHAR(255),
      ProgramName VARCHAR(255),SPID2 INT,
      REQUESTID INT)
INSERT INTO #sp_who2 EXEC sp_who2
SELECT      *
FROM        #sp_who2
WHERE       DBName = 'SystemManager7' and ProgramName = 'EntityFramework'
ORDER BY    LastBatch ASC

SELECT  c.session_id, t.text,s.program_name,
        QUOTENAME(OBJECT_SCHEMA_NAME(t.objectid, t.dbid)) + '.'
        + QUOTENAME(OBJECT_NAME(t.objectid, t.dbid)) proc_name,
        c.connect_time,
        s.last_request_start_time,
        s.last_request_end_time,
        s.status
FROM    sys.dm_exec_connections c
JOIN    sys.dm_exec_sessions s ON c.session_id = s.session_id
JOIN    #sp_who2 w on c.session_id = w.SPID
CROSS APPLY sys.dm_exec_sql_text(c.most_recent_sql_handle) t
WHERE s.program_name = 'EntityFramework'
order by last_request_end_time

DROP TABLE #sp_who2
```

# Get last sql of session

```sql
SELECT  c.session_id, t.text,s.program_name,
        QUOTENAME(OBJECT_SCHEMA_NAME(t.objectid, t.dbid)) + '.'
        + QUOTENAME(OBJECT_NAME(t.objectid, t.dbid)) proc_name,
        c.connect_time,
        s.last_request_start_time,
        s.last_request_end_time,
        s.status
FROM    sys.dm_exec_connections c
JOIN    sys.dm_exec_sessions s
        ON c.session_id = s.session_id
CROSS APPLY sys.dm_exec_sql_text(c.most_recent_sql_handle) t WHERE s.program_name = 'EntityFramework'
order by last_request_end_time
```

# Get locks full information

```sql
SELECT  L.request_session_id AS SPID,
        DB_NAME(L.resource_database_id) AS DatabaseName,
        O.Name AS LockedObjectName,
        P.object_id AS LockedObjectId,
        L.resource_type AS LockedResource,
        L.request_mode AS LockType,
        ST.text AS SqlStatementText,
        ES.login_name AS LoginName,
        ES.host_name AS HostName,
        TST.is_user_transaction as IsUserTransaction,
        AT.name as TransactionName,
        CN.auth_scheme as AuthenticationMethod
FROM    sys.dm_tran_locks L
        JOIN sys.partitions P ON P.hobt_id = L.resource_associated_entity_id
        JOIN sys.objects O ON O.object_id = P.object_id
        JOIN sys.dm_exec_sessions ES ON ES.session_id = L.request_session_id
        JOIN sys.dm_tran_session_transactions TST ON ES.session_id = TST.session_id
        JOIN sys.dm_tran_active_transactions AT ON TST.transaction_id = AT.transaction_id
        JOIN sys.dm_exec_connections CN ON CN.session_id = ES.session_id
        CROSS APPLY sys.dm_exec_sql_text(CN.most_recent_sql_handle) AS ST
WHERE   resource_database_id = db_id()
ORDER BY L.request_session_id
```